# PSNR/SSIM/LPIPS evaluation (portable)

This notebook computes PSNR, SSIM, and LPIPS on the held-out test set using renders produced by `on-the-fly-nvs`.

- Works both on Google Colab and local Windows/Linux/macOS.
- Paths are resolved dynamically; set your scene under `on-the-fly-nvs/data/my_scene` and model under `on-the-fly-nvs/results/scene`.
- Metrics CSV is saved to `results/scene/metrics_testhold_{TEST_HOLD}.csv`.

Run all cells after your model has rendered the test images into `results/scene/test_images`. 

In [ ]:
#@title 1) Clone and install on-the-fly-nvs + dependencies
!git clone --recursive https://github.com/graphdeco-inria/on-the-fly-nvs.git
%cd on-the-fly-nvs

!pip install -q -r requirements.txt

import torch
print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Cloning into 'on-the-fly-nvs'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 808 (delta 22), reused 14 (delta 13), pack-reused 773 (from 1)
Receiving objects: 100% (808/808), 5.20 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (259/259), done.
Submodule 'submodules/Depth-Anything-V2' (https://github.com/DepthAnything/Depth-Anything-V2.git) registered for path 'submodules/Depth-Anything-V2'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim) registered for path 'submodules/fused-ssim'
Submodule 'submodules/graphdecoviewer' (https://github.com/graphdeco-inria/graphdecoviewer.git) registered for path 'submodules/graphdecoviewer'
Cloning into '/content/on-the-fly-nvs/submodules/Depth-Anything-V2'...
remote: Enumerating objects: 142, done.        
remote: Total 142 (delta 0), reused 0 (delta 0), pack-reused 142 (from 1)        
Receiving objects: 100% (142/142),

In [ ]:
!pip -q install opencv-python-headless tqdm

import os, shutil, glob, re, json, subprocess, shlex, struct
from pathlib import Path
import numpy as np, cv2

# Portable paths: Colab vs local

def is_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

ROOT = Path.cwd()
REPO = "/content/on-the-fly-nvs" if is_colab() else str((ROOT / "on-the-fly-nvs").resolve())

SCENE_DIR  = f"{REPO}/data/my_scene"     # 1528 images sequence
MODEL_DIR  = f"{REPO}/results/scene"     # trained model output
TEST_HOLD  = 30                           # evaluation split (take 1 every 30)

for d in [MODEL_DIR, SCENE_DIR]:
    os.makedirs(d, exist_ok=True)

In [ ]:
images_dir = Path(SCENE_DIR) / "images"
test_dir   = Path(SCENE_DIR) / "test"
images_dir.mkdir(parents=True, exist_ok=True)

# If dataset uses pattern "frame-000000.color.jpg", normalize to ".jpg"
# without overwriting originals: copy/symlink if needed.
def canonical_name(p: Path) -> str:
    s = p.name
    if s.endswith(".color.jpg"):
        s = s.replace(".color.jpg", ".jpg")
    return s

if not test_dir.exists():
    test_dir.mkdir(parents=True, exist_ok=True)
    imgs = sorted([p for p in images_dir.iterdir() if p.suffix.lower() in [".jpg",".jpeg",".png"] or p.name.endswith(".color.jpg")])
    assert len(imgs)>0, f"No images found in {images_dir}"
    sel = [imgs[i] for i in range(0, len(imgs), TEST_HOLD)]
    print(f"Creating {len(sel)} held-out test images in: {test_dir}")
    for p in sel:
        dst = test_dir / canonical_name(p)
        if not dst.exists():
            # copy (more robust than symlink on Colab)
            dst.write_bytes(p.read_bytes())
else:
    print(f"'test/' already exists ({test_dir}).")

Création de 51 vues de test dans: /content/on-the-fly-nvs/data/my_scene/test


In [ ]:
#@title 3B) Copy init PLY (MoGE) and use repo hook
import sys, pathlib, textwrap, glob, os, shutil
PLY_PATH = str((Path.cwd() / "3dgs_ETAPE2.ply").resolve()) if (Path.cwd() / "3dgs_ETAPE2.ply").exists() else ""

# Copy the init 3DGS PLY to the conventional location so train.py picks it up
POINTCLOUD_DIR = os.path.join(MODEL_DIR, "point_cloud")
os.makedirs(POINTCLOUD_DIR, exist_ok=True)
PC_PLY = os.path.join(POINTCLOUD_DIR, "point_cloud.ply")
if PLY_PATH:
    shutil.copy(PLY_PATH, PC_PLY)
    print("Init PLY copied to:", PC_PLY)
else:
    print("No PLY found; training will fall back to geometric bootstrap.")

# Do not rewrite the hook here; rely on init_from_ply_hook.py committed in the repo
HOOK_PATH = os.path.abspath("init_from_ply_hook.py")
if os.path.exists(HOOK_PATH):
    print("Using repo hook:", HOOK_PATH)
else:
    print("WARNING: hook not found at:", HOOK_PATH)
    print("Please keep init_from_ply_hook.py at the repo root to enable starting from existing PLY.")

# From here, the on-the-fly code estimates/refines as usual; identity on frame-0 is implicit in our frame.


Init PLY copié vers: /content/on-the-fly-nvs/results/scene/point_cloud/point_cloud.ply
Hook écrit: /content/on-the-fly-nvs/init_from_ply_hook.py


In [ ]:
#@title 4) Launch incremental on-the-fly optimization
import subprocess, shlex, sys, os

# Pass the hook via PYTHONPATH so train.py can import it
env = os.environ.copy()
prev_pp = env.get("PYTHONPATH", "")
extra = os.getcwd()
env["PYTHONPATH"] = os.pathsep.join([p for p in [prev_pp, extra] if p])

# Base command (see README)
cmd = f"python train.py -s {SCENE_DIR} -m {MODEL_DIR} --test_hold {TEST_HOLD} --viewer_mode none --downsampling 2.5 --save_every 100"
print("RUN:", cmd)
ret = subprocess.call(shlex.split(cmd), env=env)
print("Exit code:", ret)


RUN: python train.py -s /content/on-the-fly-nvs/data/my_scene -m /content/on-the-fly-nvs/results/scene --test_hold 30 --viewer_mode none --downsampling 2.5  --save_every 100
Exit code: 0


In [ ]:
# @title 3) Verify sets (identical filenames)
import os, glob

GT_TEST_DIR     = f"{SCENE_DIR}/test"
RENDER_TEST_DIR = f"{MODEL_DIR}/test_images"

gt_names = sorted([os.path.basename(p) for p in glob.glob(os.path.join(GT_TEST_DIR, "*")) if os.path.isfile(p)])
rd_names = sorted([os.path.basename(p) for p in glob.glob(os.path.join(RENDER_TEST_DIR, "*")) if os.path.isfile(p)])

print(f"GT test images: {len(gt_names)}")
print(f"Rendered tests: {len(rd_names)}")

missing_in_render = [n for n in gt_names if n not in rd_names]
missing_in_gt     = [n for n in rd_names if n not in gt_names]

if not missing_in_render and not missing_in_gt:
    print("✅ Name sets match (GT test ↔ renders).")
else:
    if missing_in_render:
        print(f"⚠️ Missing in renders ({len(missing_in_render)}) e.g.: {missing_in_render[:5]}")
    if missing_in_gt:
        print(f"⚠️ Missing in GT test ({len(missing_in_gt)}) e.g.: {missing_in_gt[:5]}")

GT test images : 51
Rendered tests : 51
✅ Les ensembles de noms correspondent (GT test ↔ rendus).


In [ ]:
# === Add SSIM / LPIPS on held-out pairs (GT test/ ↔ rendered test_images/) ===
!pip -q install lpips scikit-image

import os, glob, shutil
from pathlib import Path
import cv2
import numpy as np
import torch
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import lpips

# Portable paths: Colab vs local

def is_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

ROOT = Path.cwd()
REPO = "/content/on-the-fly-nvs" if is_colab() else str((ROOT / "on-the-fly-nvs").resolve())

SCENE_DIR  = f"{REPO}/data/my_scene"          # contains images/ and test/
MODEL_DIR  = f"{REPO}/results/scene"          # contains test_images/ (renders)
GT_TEST_DIR     = f"{SCENE_DIR}/test"
RENDER_TEST_DIR = f"{MODEL_DIR}/test_images"
CSV_PATH        = f"{MODEL_DIR}/metrics_testhold_{TEST_HOLD}.csv"  # adjust if TEST_HOLD changes

# --- Utilities ---
def read_rgb(path):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None: return None
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

def psnr_tensor(img01, gt01):
    """
    img01, gt01: torch.FloatTensor [C,H,W] in [0,1]
    Return: PSNR (dB) with MAX=1 => PSNR = -10*log10(MSE)
    """
    diff = (img01 - gt01).reshape(3, -1)
    mse  = torch.mean(diff**2)
    if mse <= 1e-12:
        return float("inf")
    return float(-10.0 * torch.log10(mse).item())

def ssim_y_channel(img01_np, gt01_np):
    """
    img01_np, gt01_np: numpy uint8 or float32 [H,W,3]
    Convert to Y (luma) then compute SSIM on [0,1]
    """
    if img01_np.dtype != np.uint8:
        # if already [0,1], cast to uint8 for OpenCV YCrCb conversion
        img8 = np.clip((img01_np * 255.0 + 0.5), 0, 255).astype(np.uint8)
    else:
        img8 = img01_np
    if gt01_np.dtype != np.uint8:
        gt8  = np.clip((gt01_np * 255.0 + 0.5), 0, 255).astype(np.uint8)
    else:
        gt8  = gt01_np

    img_y = cv2.cvtColor(img8, cv2.COLOR_RGB2YCrCb)[:,:,0].astype(np.float32) / 255.0
    gt_y  = cv2.cvtColor(gt8,  cv2.COLOR_RGB2YCrCb)[:,:,0].astype(np.float32) / 255.0
    return float(ssim(gt_y, img_y, data_range=1.0))

def to_lpips_tensor_u8(rgb_u8):
    """
    rgb_u8: numpy uint8 [H,W,3] RGB
    Return: torch.FloatTensor [1,3,H,W] in [-1,1], contiguous (avoids negative strides)
    """
    t = torch.from_numpy(rgb_u8.copy())  # .copy() to avoid negative strides after slicing
    t = t.permute(2,0,1).float().unsqueeze(0)  # [1,3,H,W]
    t = t / 127.5 - 1.0
    return t

# --- List rendered (target) filenames ---
rd_names = sorted([os.path.basename(p) for p in glob.glob(os.path.join(RENDER_TEST_DIR, "*"))
                   if os.path.isfile(p)])
assert rd_names, f"No renders found in {RENDER_TEST_DIR}"

# --- Build aligned pairs (GT ↔ Render) ---
pairs = []
for name in rd_names:
    gt_p = os.path.join(GT_TEST_DIR, name)
    rd_p = os.path.join(RENDER_TEST_DIR, name)
    if os.path.isfile(gt_p) and os.path.isfile(rd_p):
        pairs.append((gt_p, rd_p))
print(f"Pairs ready: {len(pairs)}")

# --- Prepare LPIPS ---
lpips_net = lpips.LPIPS(net='vgg')
if torch.cuda.is_available():
    lpips_net = lpips_net.cuda()
lpips_net.eval()

# --- Metrics loop ---
psnrs, ssim_list, lpips_list = [], [], []
rows = ["name,psnr_db,ssim_y,lpips_vgg"]

for i, (gt_p, rd_p) in enumerate(tqdm(pairs, desc="Metrics")):
    gt = read_rgb(gt_p); rd = read_rgb(rd_p)
    if gt is None or rd is None:
        continue

    # Ensure same spatial size (like in PSNR code)
    if rd.shape != gt.shape:
        rd = cv2.resize(rd, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_AREA)

    # --- PSNR on [0,1]
    gt_t = torch.from_numpy(gt).permute(2,0,1).float() / 255.0
    rd_t = torch.from_numpy(rd).permute(2,0,1).float() / 255.0
    val_psnr = psnr_tensor(rd_t, gt_t)

    # --- SSIM on Y channel
    val_ssim = ssim_y_channel(rd, gt)

    # --- LPIPS (RGB, [-1,1])
    t_gt = to_lpips_tensor_u8(gt)
    t_rd = to_lpips_tensor_u8(rd)
    if torch.cuda.is_available():
        t_gt = t_gt.cuda(non_blocking=True)
        t_rd = t_rd.cuda(non_blocking=True)
    with torch.no_grad():
        val_lpips = float(lpips_net(t_gt, t_rd).item())

    psnrs.append(val_psnr); ssim_list.append(val_ssim); lpips_list.append(val_lpips)
    rows.append(f"{os.path.basename(gt_p)},{val_psnr:.4f},{val_ssim:.4f},{val_lpips:.4f}")

    if i < 5 or i % 10 == 0:
        print(f"[{i:03d}] {os.path.basename(gt_p)} : PSNR={val_psnr:.2f} dB | SSIM(Y)={val_ssim:.3f} | LPIPS={val_lpips:.3f}")

# --- Summary + CSV ---
if psnrs:
    arrP = np.array(psnrs, dtype=np.float64)
    arrS = np.array(ssim_list, dtype=np.float64)
    arrL = np.array(lpips_list, dtype=np.float64)
    print(f"\nMeans ({len(psnrs)} views) → "
          f"PSNR={arrP.mean():.2f} dB | SSIM(Y)={arrS.mean():.3f} | LPIPS={arrL.mean():.3f}")
    print(f"  Medians → PSNR={np.median(arrP):.2f} | SSIM(Y)={np.median(arrS):.3f} | LPIPS={np.median(arrL):.3f}")
    print(f"  Min/Max → PSNR=({arrP.min():.2f},{arrP.max():.2f}) | "
          f"SSIM=({arrS.min():.3f},{arrS.max():.3f}) | LPIPS=({arrL.min():.3f},{arrL.max():.3f})")

    with open(CSV_PATH, "w") as f:
        f.write("\n".join(rows))
    print(f"\nCSV written: {CSV_PATH}")
else:
    print("No pairs evaluated — check paths and filenames.")

Paires prêtes: 51
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth


Metrics:   2%|▏         | 1/51 [00:00<00:28,  1.72it/s]

[000] frame-000000.jpg : PSNR=28.74 dB | SSIM(Y)=0.727 | LPIPS=0.609


...

[030] frame-000900.jpg : PSNR=19.47 dB | SSIM(Y)=0.612 | LPIPS=0.627


Metrics:  80%|████████  | 41/51 [00:20<00:04,  2.05it/s]

[040] frame-001200.jpg : PSNR=23.58 dB | SSIM(Y)=0.618 | LPIPS=0.595


Metrics: 100%|██████████| 51/51 [00:24<00:00,  2.05it/s]

[050] frame-001500.jpg : PSNR=19.81 dB | SSIM(Y)=0.639 | LPIPS=0.646

Moyennes (51 vues) → PSNR=21.91 dB | SSIM(Y)=0.620 | LPIPS=0.633
  Médianes → PSNR=21.11 | SSIM(Y)=0.618 | LPIPS=0.639
  Min/Max  → PSNR=(16.53,29.01) | SSIM=(0.515,0.778) | LPIPS=(0.546,0.709)

CSV écrit : /content/on-the-fly-nvs/results/scene/metrics_testhold_30.csv


In [ ]:
import os
import pandas as pd
from pathlib import Path

# Resolve CSV path dynamically (works on Colab and local)
try:
    csv_path = f"{MODEL_DIR}/metrics_testhold_{TEST_HOLD}.csv"
except NameError:
    # Fallback if previous cells were not run
    def is_colab():
        return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ
    ROOT = Path.cwd()
    REPO = "/content/on-the-fly-nvs" if is_colab() else str((ROOT / "on-the-fly-nvs").resolve())
    MODEL_DIR = f"{REPO}/results/scene"
    TEST_HOLD = 30
    csv_path = f"{MODEL_DIR}/metrics_testhold_{TEST_HOLD}.csv"

print("Reading:", csv_path)
df = pd.read_csv(csv_path)
df

,name,psnr_db,ssim_y,lpips_vgg
0,frame-000000.jpg,28.7400,0.7275,0.6094
1,frame-000030.jpg,27.3659,0.6397,0.5456
2,frame-000060.jpg,28.3334,0.6703,0.5887
3,frame-000090.jpg,27.5664,0.6945,0.5583
4,frame-000120.jpg,23.1522,0.5969,0.6265
5,frame-000150.jpg,29.0071,0.6963,0.5989
6,frame-000180.jpg,27.6926,0.6555,0.5585
7,frame-000210.jpg,22.4407,0.5960,0.5953
8,frame-000240.jpg,17.6212,0.5511,0.6628
9,frame-000270.jpg,18.7323,0.5323,0.6493
